# Visualisation of LFP & Sleep scoring

### Import recordings

Load packages

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
import ephyviewer
from scipy.stats import zscore
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json
import IPython
from matplotlib import cm
from matplotlib.colors import to_hex
import ast
from ephyviewer import AnalogSignalSourceWithScatter

%matplotlib widget

In [2]:
#Load LFP coordinates 
notebook_path = Path("/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]))
Channels = f'{notebook_path.parent}/_LFP_coordinates_of_all_mice.csv'
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)

Choose OpenEphys folder

In [3]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=dpath.parent
except:
    print("the path was not defined in store")
    dpath = "//10.69.168.1/crnldata/forgetting/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\Analysed2025_AB\L1NDNF_mice\BlackLines\ba…

### Load LFPs data 

In [4]:
folder_base = Path(dpath) 

if Path(f'{folder_base}/DataFrame_rawdataDS.pkl').exists(): # prefer loading downsample file over original file
    print('DataFrame_rawdataDS.pkl file')
    LFPfile = Path(f'{folder_base}/DataFrame_rawdataDS.pkl')
    LFPs_df = pd.read_pickle(LFPfile)
    samplerate = 1000 
    numchannel = LFPs_df.shape[1]
    rec_ch_list = LFPs_df.columns.values
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
elif Path(f'{folder_base}/RawDataChannelExtractedDS.npy').exists():
    print('RawDataChannelExtractedDS.npy file')
    LFPfile = Path(f'{folder_base}/RawDataChannelExtractedDS.npy')
    DataRec = np.load(LFPfile, mmap_mode= 'r')
    samplerate = 1000 
    numchannel = DataRec.shape[1]
    rec_ch_list = np.arange(0,numchannel,1)
    LFPs_df=pd.DataFrame(DataRec, columns=rec_ch_list)
    print('No metadata found')
elif Path(f'{folder_base}/continuous.dat').exists():
    print('continuous.dat file')
    LFPfile = Path(f'{folder_base}/continuous.dat')
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    #filepath = Path(os.path.join(folder_base, f'structure.oebin'))
    with open(filepath) as f:
        metadata = json.load(f)
    samplerate = metadata['continuous'][0]['sample_rate']  
    numchannel = metadata['continuous'][0]['num_channels'] 
    rec_ch_list = np.array([int(''.join(c for c in metadata['continuous'][0]['channels'][x]['channel_name'] if c.isdigit()))-1 for x in range(0, len(metadata['continuous'][0]['channels']))])
    DataRec = DataRec.reshape(-1,numchannel)
    print('Metadata found')
    LFPs_df=pd.DataFrame(DataRec, columns=rec_ch_list) 
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS) 
else: 
    print('no LFPs file found')

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(LFPs_df.shape[0]/samplerate/60), 'min of recording')

DataFrame_rawdataDS.pkl file
sample rate = 1000 Hz
32 channels recorded
16 min of recording


Downsample LFP data to 1kHz if needed & save it

In [ ]:
if samplerate > 1000:
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df)[0] * new_sampling_rate / samplerate)
    LFPs_df_DS = pd.DataFrame(signal.resample(LFPs_df, Nmber_points, axis = 0), columns=LFPs_df.columns.values)
    LFPtimestampsDS = LFPtimestamps[::int(samplerate/new_sampling_rate)][:-1]
    samplerate = new_sampling_rate
    LFPs_df_DS.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    LFPs_df = LFPs_df_DS
    LFPtimestamps = LFPtimestampsDS
# eventually delete original files to gain space

Identify mouse

In [5]:
mouse = []
pos_mice = []
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mice.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mice, mouse))] # sort mouse in the same order as they appear in the path

if len(mouse) > 1: # found multiple mouse name in the path
    if max(rec_ch_list) <= 31: # no channels superior to 32, so only one mouse recorded
        id = 1 # change to 0 to see the first mouse name found, 1 the second, etc
        ID = 0
        print(f"/!/ Mutliple mice name found in the path but only mouse recorded = {mouse}. The n°{id+1} was choosen automatically = {mouse[id]}.")
        mouse = mouse[id]
    else:
        ###################################################################################################################################
        ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        ###################################################################################################################################
        print(f"/!/ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse = mouse[ID] 
elif max(rec_ch_list) > 31 & len(mouse) == 1:
    ###################################################################################################################################
    ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
    ###################################################################################################################################
    mouse = mouse[0]
    print(f"/!/ Mutliple mice recorded at the same time, but only one found in the path = {mouse}. The n°{ID+1} was choosen automatically = {mouse}.")
elif len(mouse) == 1: # found only one mouse name in the path
    ID = 0
    mouse = mouse[ID]

/!/ Mutliple mice recorded at the same time, but only one found in the path = BlackLines. The n°1 was choosen automatically = BlackLines.


Identify electrodes & create differential LFPs

In [6]:
all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []

RecordedArea = []
ChoosenChannels = []
combined = []
if mouse:
    rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
    for rec_ch in rec_ch_list_mouse:
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region = all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str != 'nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
                if len(num_ch) > 0:
                    region = all_LFPcoordinates.loc[mouse].index[idx]
                    LFP = locals()[region]
                    LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                    locals()[region] = LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP = locals()[region]
        LFP_ch = locals()[f'{region}_ch']
        if len(LFP) > 0:
            combined = zscore(LFP[:,np.newaxis]) if len(combined) == 0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!/ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse = '' # fill mouse name
    RecordedArea = ['EMG','S1','PFC','CA1'] 
    EMG = LFPs_df[0]
    S1 = LFPs_df[0]-LFPs_df[1]
    combined = np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

BlackLines
['EMG', 'S1', 'PFC', 'CA1']
[[26], [2, 3], [5, 6], [28, 30]]


### Score sleep manually



4 stages : Wake, NREM, REM, undefined

In [ ]:
labels = ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] # Wake, NREM sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 20 # default window length in sec

SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize'] = winlen
view2.params['new_epoch_step'] = epoch_dur

view2.by_label_params['label0', 'color'] = '#ffcd69' #AW
view2.by_label_params['label1', 'color'] = '#69cfff' #NREM
view2.by_label_params['label2', 'color'] = '#ff69cd' #REM
view2.by_label_params['label3', 'color'] = '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
#addItem.ctrl.logXCheck.isChecked()

for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

6 stages : AW, QW, NREM, IS, REM, undefined

In [7]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 20 # default window length in sec

SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view2.params['xsize'] = winlen
view2.params['new_epoch_step'] = epoch_dur
view2.by_label_params['label0', 'color'] = '#ffcd69' #AW
view2.by_label_params['label1', 'color'] = '#69ffe6' #QW
view2.by_label_params['label2', 'color'] = '#69cfff' #NREM
view2.by_label_params['label3', 'color'] = '#cd69ff' #IS
view2.by_label_params['label4', 'color'] = '#ff69cd' #REM
view2.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

0

### Check all channels

In [ ]:
app = mkQApp()
winlen = 20 # default window length in sec
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
view1 = TraceViewer.from_numpy(np.array(LFPs_df), samplerate, t_start, 'Signals')

cmap = cm.Spectral
values = np.linspace(0, 1, np.array(LFPs_df).shape[1])
colormap = [to_hex(rgb) for rgb in cmap(values)]
for idx in range(np.array(LFPs_df).shape[1]): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx]

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.params['xsize'] = winlen
win.add_view(view1)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)

#Run
win.show()
app.exec()


### Flip omnetics headstage

Run the cell bellow if you believe that the omnetic headstage was reverted for this mouse during this recording (if you run it twice it will go back to the initial configuration).

In [ ]:
rec_ch_list_ID = LFPs_df.columns-ID*32
rec_ch_list_mouse = [value for value in rec_ch_list_ID if 0 <= value <= 31]
i = np.argmax(rec_ch_list_ID>=0)
inverted_chs = np.concatenate([range(16,32,1), range(0,16,1)], axis=0)
LFPs_df_mouse=LFPs_df.iloc[:,i:i+len(rec_ch_list_mouse)]
flipped_ch=(inverted_chs[LFPs_df_mouse.columns-(ID*32)])+(ID*32)
LFPs_df.columns.values[i:i+len(rec_ch_list_mouse)] = flipped_ch
LFPs_df = LFPs_df.sort_index(axis=1)

# Reselect electrodes
all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []
    
RecordedArea = []
ChoosenChannels = []
combined = []
if mouse:
    rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
    for rec_ch in rec_ch_list_mouse:
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region = all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str != 'nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
                if len(num_ch) > 0:
                    region = all_LFPcoordinates.loc[mouse].index[idx]
                    LFP = locals()[region]
                    LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                    locals()[region] = LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP = locals()[region]
        LFP_ch = locals()[f'{region}_ch']
        if len(LFP) > 0:
            combined = zscore(LFP[:,np.newaxis]) if len(combined) == 0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!/ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse = '' # fill mouse name
    RecordedArea = ['EMG','S1','PFC','CA1'] 
    EMG = LFPs_df[0]
    S1 = LFPs_df[0]-LFPs_df[1]
    combined = np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

Save if you're now sure that the omnetic headstage was reverted for this mouse during this recording

In [ ]:
if samplerate <= 1000:
    LFPs_df.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
else: # best to downsample too
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df)[0] * new_sampling_rate / samplerate)
    LFPs_df_DS = pd.DataFrame(signal.resample(LFPs_df, Nmber_points, axis = 0), columns=LFPs_df.columns.values)
    LFPs_df_DS.iloc[:,33:65].to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
    #LFPs_df_DS.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')

Save a portion of the signals

In [ ]:
#LFPs_df.iloc[:,33:65].to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
LFPs_df.iloc[:,0:32].to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')

Save the signals with timestamps

In [ ]:
if samplerate <= 1000:
    LFPs_df.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
else: # best to downsample too
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df)[0] * new_sampling_rate / samplerate)
    LFPs_df_DS = pd.DataFrame(signal.resample(LFPs_df, Nmber_points, axis = 0), columns=LFPs_df.columns.values)
    LFPs_df_DS.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
    
    LFPtimestampsDS = LFPtimestamps[::int(samplerate/new_sampling_rate)][:-1]
    samplerate = new_sampling_rate
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    LFPs_df = LFPs_df_DS
    LFPtimestamps = LFPtimestampsDS

### Visualize detected oscillations 

Upload detected oscillation

In [14]:
# SWR     
SWSdetection=folder_base / f'SWR_detection.csv'
#SWSdetection=folder_base / f'SWRproperties.csv'
SWR_prop = pd.read_csv(SWSdetection, index_col=0)  
#SWR_prop['toKeep'] = SWR_prop['toKeep'].astype(str)
#SWR_prop  = SWR_prop[SWR_prop['toKeep'].isin(['VRAI', 'True'])]   
SWR_start = np.transpose((SWR_prop['start time']).astype(int))
SWR_end = np.transpose((SWR_prop['end time']).astype(int))

# Spindles
Spdldetection=folder_base / f'SpindlesS1&PFC_detection.csv'
#Spdldetection=folder_base / f'Spindleproperties_S1&PFC.csv'
Spdl_prop = pd.read_csv(Spdldetection, index_col=0)  
#Spdl_prop['toKeep'] = Spdl_prop['toKeep'].astype(str)
#Spdl_prop  = Spdl_prop[Spdl_prop['toKeep'].isin(['VRAI', 'True'])]   

S1Spdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1']
S1spdl_start = np.transpose(S1Spdl_prop['start time'].astype(int))
S1spdl_end = np.transpose(S1Spdl_prop['end time'].astype(int))

PFCSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='PFC']
PFCspdl_start = np.transpose(PFCSpdl_prop['start time'].astype(int))
PFCspdl_end = np.transpose(PFCSpdl_prop['end time'].astype(int))

GlobalSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1PFC']
GBLspdl_start = np.transpose(GlobalSpdl_prop['start time'].astype(int))
GBLspdl_end = np.transpose(GlobalSpdl_prop['end time'].astype(int))

print(f'{len(SWR_end)} SWR in CA1, {len(S1spdl_end)} Spdl in S1, {len(PFCspdl_end)} Spdl in PFC, {len(GBLspdl_end)} Global Spdl')

137 SWR in CA1, 26 Spdl in S1, 1 Spdl in PFC, 8 Global Spdl


In [15]:
app = mkQApp()
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 10 # default window length in sec

try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: GBLspdl_start, 5: GBLspdl_end, 6: SWR_start, 7: SWR_end}
    scatter_channels = {0: [2], 1: [2], 2: [1], 3: [1], 4: [1,2], 5: [1,2], 6: [3], 7: [3]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#BBBBBB', 5: '#555555', 6: '#FFFFFF', 7: '#222222'}, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()


# FFT
view2 = TimeFreqViewer(source=source, name='FFT')

view2.params['show_axis'] = True
view2.params['timefreq', 'f_start'] = 1
view2.params['timefreq', 'f_stop'] = 200
view2.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view2.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view2.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view2.by_channel_params[f'ch{idx}', 'clim'] = 1
        view2.by_channel_params[f'ch{idx}', 'visible'] = True


# Scoring 
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)
view3 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view3.params['xsize'] = winlen
view3.params['new_epoch_step'] = epoch_dur
view3.by_label_params['label0', 'color'] = '#ffcd69' #AW
view3.by_label_params['label1', 'color'] = '#69ffe6' #QW
view3.by_label_params['label2', 'color'] = '#69cfff' #NREM
view3.by_label_params['label3', 'color'] = '#cd69ff' #IS
view3.by_label_params['label4', 'color'] = '#ff69cd' #REM
view3.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
view3.controls.hide()

#Run
win.add_view(view1)
win.add_view(view2)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()


viewer has moved already 1 29.0 28.0
viewer has moved already 3 29.0 28.0
viewer has moved already 2 29.0 28.0
viewer has moved already 2 38.0 33.0
viewer has moved already 2 38.0 34.0
viewer has moved already 2 38.0 35.0
viewer has moved already 2 38.0 36.0
viewer has moved already 2 38.0 37.0
viewer has moved already 1 38.0 33.0
viewer has moved already 1 38.0 34.0
viewer has moved already 1 38.0 35.0
viewer has moved already 3 38.0 33.0
viewer has moved already 1 38.0 36.0
viewer has moved already 3 38.0 34.0
viewer has moved already 3 38.0 35.0
viewer has moved already 1 38.0 37.0
viewer has moved already 3 38.0 36.0
viewer has moved already 3 38.0 37.0
viewer has moved already 2 40.0 39.0
viewer has moved already 3 40.0 39.0
viewer has moved already 1 40.0 39.0
viewer has moved already 2 42.0 41.0
viewer has moved already 1 42.0 41.0
viewer has moved already 3 42.0 41.0
viewer has moved already 3 44.0 43.0
viewer has moved already 2 44.0 43.0
viewer has moved already 1 44.0 43.0
v

0